In [792]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 300
pd.options.display.max_columns = 999

In [793]:
#Read data
df = pd.read_csv("../../Data/Tidy_Full.csv", index_col = 0)

In [794]:
#Save DQs before removing any information
dq_indexes = df[df.apply(lambda r: r.str.contains('(BMB|None|DQ|Disq)', case=False).any(), axis=1)].index

C:\Users\Joe\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [795]:
#Inspect data
df.head()

,CompID,Name,Sex,Class,Division,BestBP,Total,Place,BestSQ,BestDL,BDW,BP1,BP2,BP3,Wilks,SQ1,SQ2,SQ3,DL1,DL2,DL3,DOB
Index,,,,,,,,,,,,,,,,,,,,,,
0,1,Barbara Hamilton,F,56,O/50,42.5/43,42.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Nat Zacharko,F,56,O/40,60,60,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Dean Jenkinson,M,67.5,Open,125,125,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Aaron McFarlane,M,67.5,Open,105,105,2-Jan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,David Graham,M,67.5,O/60,52.5,52.5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Goals:
- Homogenize column values
- Fix data types
- Change no-lifts to minus (rather than x suffix)

##### Sex column

In [796]:
#Inspect
df.Sex.value_counts()

M           15488
F            6969
M-O            63
M-J            43
m              42
F-O            18
F-J            17
f              16
M-M1           16
M - Open       12
M-SJ           11
M-M3           11
F-M1           11
F - M1          7
F - Open        6
M - O           6
M-M2            6
M-Sub           5
W               4
M - M2          3
M-BL            2
F - M2          2
F-SJ            2
F-M3            2
F-M4            1
M - U18         1
M-Jnr           1
M - M1          1
F-M2            1
M - SJ          1
Name: Sex, dtype: int64

In [797]:
#Fix
def fix_sex(df):
    df.Sex = df.Sex.replace({'m': 'M', 'f': 'F', 'W': 'F'})
    df.Sex = df.Sex.str.replace(' ','')
    df[df.Sex.str.contains('-')].Division.isnull().all() #All rows with div info in sex column have nulls in div column
    df.Division.fillna(pd.DataFrame(df.Sex.str.split('-').to_list(), columns = ['Sex', 'Division'])['Division'], inplace = True)
    df.Sex = pd.DataFrame(df.Sex.str.split('-').to_list(), columns = ['Sex', 'Division'])['Sex']
    return df

##### Class column

In [798]:
#Inspect
#df[~df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts() #Possibly bad values
df[df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts() #Okay values
#df[df.Class.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Class.value_counts().plot(kind='bar') #Histogram of okay values

82.5      846
90        767
75        735
100       598
67.5      504
83        483
93        473
110       459
105       447
125       390
74        315
120       261
125+      238
60        228
56        173
120+      167
66        166
52        162
63        145
72        121
57        110
84         91
84+        90
90+        85
48         56
59         42
47         37
44          5
75.5        1
92.54       1
55.9        1
120.8       1
89.2        1
102         1
76.4        1
67.6        1
117.26      1
66.9        1
89.6        1
62.5        1
Name: Class, dtype: int64

In [799]:
#Fix
def fix_class(df):
    df.Class = df.Class.str.replace('kg', '')
    df.Class = df.Class.str.replace('x', '')
    df.Class = df.Class.str.replace(' ', '')
    df[df.Class.fillna('').str.contains('-')].Division.isnull().all() #All rows with div info in class column have nulls in div column
    df.Division.fillna(pd.DataFrame(df.Class.fillna('').str.split('-').to_list(), columns = ['Class', 'Division'])['Division'], inplace = True)
    df.Class = pd.DataFrame(df.Class.fillna('').str.split('-').to_list(), columns = ['Class', 'Division'])['Class']
    df.Class.replace('', np.nan, inplace = True)
    #Look up special case comp tables to check for errors
    df.replace({'Class' : { '125.00+' : '125+', '66.9': '67.5', '55.9': '56', '62.5': '67.5', '67.6': '67.5', '75.5': '75'}}, inplace = True)
    for i in [1338, 1556, 1558, 3446, 3473, 7510, 7528]:
        temp = df.loc[i, 'BDW']
        df.loc[i, 'BDW'] = df.loc[i, 'Class']
        df.loc[i, 'Class'] = temp
    return df

##### DOB column

In [800]:
#Inspect
df.DOB.value_counts()
#set([k[0] for j, k in df[df.DOB.notnull()].iterrows() if any(substring in k[21] for substring in ['PNG', 'NRU', '????', 'DQ', '-', 'NAU', 'NZPF', 'TUV', '???', 'PRC', 'NZ', 'NIU'])])

1993     942
1992     813
1994     811
1995     772
1991     626
1989     619
1990     616
1996     614
1997     545
1987     491
1988     465
1986     364
1998     356
1982     344
1985     317
1984     305
1981     292
1983     277
1980     256
1999     236
1979     204
2000     201
1977     187
1975     166
1978     151
1970     149
1973     149
1974     142
2001     136
1976     126
2002     123
1969     122
1965     117
1966     113
1971     106
1967     102
1968     102
1972     101
1964      95
1961      90
1962      85
1956      79
1948      74
OPEN      69
1963      65
1958      62
1960      53
2003      52
1959      46
1957      42
1955      41
1949      37
2004      35
1945      32
1954      30
1950      29
1947      29
1953      28
O/40      27
1952      26
23        24
U/23      23
O         22
28        20
1944      20
O/60      19
1951      16
0         16
25        15
O/50      15
1939      15
27        14
26        14
PNG       13
NRU       12
35        11
1941      11

In [801]:
#Fix
def fix_dob(df):
    #Division in DOB column
    for i in [103, 109, 113, 114, 141, 142, 144, 146, 168, 177, 184, 214]:
        df.loc[df.CompID == i, 'Division'] = df.loc[df.CompID == i, 'DOB']
        df.loc[df.CompID == i, 'DOB'] = np.nan
    #Words in DOB column
    df.DOB.replace(['PNG', 'NRU', '????', 'DQ', '-', 'NAU', 'NZPF', 'TUV', '???', 'PRC', 'NZ', 'NIU'], np.nan, inplace = True)
    df.DOB.replace('`987', 1987, inplace = True)
    #Big numbers
    df.DOB.replace('\d{5}', np.nan, inplace = True, regex = True)
    #Small numbers
    df.replace({'DOB' : { '0' : np.nan, '1886': 1986, '1680': 1980, '995': 1995}}, inplace = True)
    df.loc[17261, 'DOB'] = 2018-30; df.loc[12972, 'DOB'] = 2016-35; df.loc[12975, 'DOB'] = 2018-30; df.loc[18635, 'DOB'] = 2018-27
    for i,j in zip([572, 574, 606, 612, 665, 939, 940], [2015, 2015, 2016, 2016, 2017, 2020, 2020]):
        df.loc[df.CompID == i, 'DOB'] = j-df.loc[df.CompID == i, 'DOB']
    #Too close to current year
    df.replace({'DOB' : { 2014 : np.nan, 2016: np.nan, 2020: np.nan}}, inplace = True)
    df.loc[12166, 'DOB'] = np.nan
    return df

##### Division column

In [802]:
#Inspect
df.Division.value_counts()
#Check rows with weird entries
#set([k[0] for j, k in df[df.Division.notnull()].iterrows() if any(substring in k[4] for substring in ['"'])])

O              1887
Open            681
J               503
M1              498
M2              310
SJ              212
M3              192
O/40            160
OPEN            131
"               114
O/50             90
O/60             89
U/23             80
Jnr              54
M4               50
VIC              35
Op               31
JNR              30
U18              26
o                23
U/19             21
BP               20
Classic          20
Jun              20
MI               18
Bp               17
O-O              17
Junior           17
U23              16
Nov              15
open             15
SJNR             15
SUB JNR          13
O-O/40           12
U/18             11
M                10
JR                9
0/50              9
O/70              8
Guest             8
MII               8
b                 8
0/40              7
OVER 40           7
MIII              7
Uni               7
OPEN?             7
Op-O/40           6
APFC              6
Oceania           6


In [803]:
#Fix
def fix_division(df):
    df.loc[df.CompID == 64, 'Division'] = df[df.CompID == 64].Division.str.replace('"', 'Snr')
    df.Division = df.Division.str.replace('"', 'Open')
    divsemdf = pd.read_csv("../../Data/Division_semantics.csv", engine = 'python')
    #Map words to semantic meaning
    semdict = {tuple(divsemdf[divsemdf[col].notnull()][col]): col for col in divsemdf.columns}
    semmap = {}
    for k, v in semdict.items():
        for key in k:
            semmap[key] = v
    #Change words
    df.Division.replace(semmap, inplace = True)
    df.Division.replace('Unknown', np.nan, inplace = True)
    df.loc[[5546, 3277], 'Division'] = 'MI'
    return df

##### Wilks column

In [804]:
#Inspect
#df.Wilks.value_counts()
df[~df.Wilks.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].Wilks.value_counts()
#df[df.Wilks.str.match('\.$') == True] #Check none start with .
#df[df.Wilks.str.match('^\.') == True] #Check none end with .

BMB       396
-         368
x           6
out         5
Disq.       4
         ... 
302,95      1
214,52      1
83,14       1
314,19      1
392,98      1
Name: Wilks, Length: 647, dtype: int64

In [805]:
#Fix
def fix_wilks(df):
    df.Wilks.replace(['BMB', '-', 'out', 'x', 'Disq.', 'Disq', '?', 'DQ (fees)', 'withdrawn', 'DQ'], np.nan, inplace = True)
    df.Wilks = df.Wilks.str.replace(',', '.', regex = False)
    return df

##### BDW column

In [806]:
#Inspect
#df.BDW.value_counts()
df[~df.BDW.fillna('').str.match(r'^-?\d+(\.\d+)?[+]?$')].BDW.value_counts()

92,20     5
81,85     5
73,85     5
65,75     4
80,80     4
         ..
100,15    1
56,54     1
61,10     1
96,10     1
73,09     1
Name: BDW, Length: 490, dtype: int64

In [807]:
#Fix
def fix_bdw(df):
    df.BDW = df.BDW.str.replace(',', '.', regex = False)
    df.BDW = df.BDW.str.replace('..', '.', regex = False)
    df.BDW = df.BDW.str.replace('?', '', regex = False)
    df.BDW.replace('BMB', np.nan, inplace = True)
    return df

##### Place column

In [808]:
#Inspect
df.Place.value_counts()
#df[df.Place.fillna('1').str.isnumeric()].Place.value_counts() #Fine
#df[~df.Place.fillna('1').str.isnumeric()].Place.value_counts() #Bad

#Bad categories:
#df[~df.Place.fillna('').str.contains('\d')].Place.value_counts() #No info
#df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].Place.value_counts() #Suffix + Additional info included
#df[df.Place.fillna('').str.match('.+(kg$)')] #All of these have division/class info already - can remove from place column
#df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)$')].Place.value_counts() #Has "th", "st", "nd", "rd" suffix
#df[df.Place.fillna('').str.contains('[a-zA-Z]')].Place.value_counts() #Additional info (from one comp only)
#df[~df.Place.fillna('1').str.isnumeric()].CompID.unique() #Comps with placing for 2 classes

1                7661
2                3571
3                2033
4                1209
5                 726
6                 459
7                 259
8                 172
-                 148
9                 111
10                 74
1st                61
11                 58
?-?                47
12                 44
GOLD               43
BMB                43
2nd                42
-?                 38
1 & 1              30
13                 30
3rd                28
14                 20
4th                20
SILVER             17
15                 16
5th                15
BRONZE             11
16                 11
7th                10
x                   8
19                  8
6th                 8
18                  8
17                  8
12th                7
9th                 7
20                  6
10th                6
1.1                 5
21                  5
22                  5
1 & 2               4
1st J 66kg          4
1st M1 120+kg       4
Disq.     

In [809]:
#Fix
def fix_place(df):
    df.Place.replace(['0', '-', '?-?', '-?', 'BMB', 'x', 'DQ', 'Disq.', '?', 'Inv', '?-??', 'X', 'Guest', 'N.E.', 'Most Imp', 'G', 'Disq', 'bmb'], np.nan, inplace = True)
    df.replace({'Place' : {'GOLD': '1', 'SILVER': '2', 'BRONZE': '3', '`.1': '1', '`1': '1'}}, inplace = True)
    df.Place = df.Place.str.replace('Jan', '1', regex = False)
    df.Place = df.Place.str.replace('Feb', '2', regex = False)
    df.Place = df.Place.str.replace('Mar', '3', regex = False)
    df.Place = df.Place.str.replace('-Nov', '', regex = False)
    df.Place = df.Place.str.replace('?', ' ', regex = False)
    df.loc[df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].index, 'Place'] = df[df.Place.fillna('').str.match('^\d+(?:st|nd|rd|th)')].Place.str.split(' ', expand = True)[0].str.strip('stnrdh') #Terrible
    df.loc[df[df.Place.fillna('').str.contains('[a-zA-Z]')].index, 'Place'] = '1'
    df.loc[df[~df.Place.fillna('1').str.isnumeric()].index, 'Place'] = np.nan
    return df

##### Name column

In [810]:
#Inspect
#df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].Name #Names that have characters other than a-z, spaces, apostrophe and dash.
#df[df.Name.str.contains("\s$")].Name
#df[df.Name.str.contains("- ")].Name

In [811]:
#Fix
def fix_name(df):
    df.loc[12467, 'Name'] = 'Steven Melekiola'
    df.loc[2036, 'Name'] = "MAX O'TOOLE"
    df.loc[1491, 'Name'] = "Mark Shillington"
    df.loc[2706, 'Name'] = "MARK O'SHEA"
    df.loc[7688, 'Name'] = "Clair Angel"
    df.loc[3088, 'Name'] = "Rick Lewis"
    df.Name = df.Name.str.replace('?', '', regex = False)
    df.Name = df.Name.str.replace("'Sasha'", '', regex = False)
    df.Name = df.Name.str.replace('*', '', regex = False)
    df.Name = df.Name.str.replace('- ', '-', regex = False)
    df.Name = df.Name.str.replace(' -', '-', regex = False)
    df.Name = df.Name.str.replace('4th BP', '', regex = False)
    df.Name = df.Name.str.replace(' - u18', '', regex = False)
    df.Name = df.Name.str.replace('u18', '', regex = False)
    df.Name = df.Name.str.replace('??QLD', '', regex = False)
    df.Name = df.Name.str.replace('QLD', '', regex = False)
    df.Name = df.Name.str.replace(r'\([^)]*\)', '')
    df.Name = df.Name.str.replace(r'\"[^)]*\"', '')
    df.loc[df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].index, 'Name'] = df[~df.Name.str.contains("^[.a-zA-Z '-]+$")].Name.str.split(', ').str[::-1].str.join(' ')
    df.Name = df.Name.str.replace('(\s{2})', '')
    df.Name = df.Name.str.strip()
    return df

##### Total column

In [812]:
#Inspect
#df.Total.value_counts() #1380
#df[df.Total.fillna('').str.contains("[a-zA-Z]")].Total.value_counts()
df[df.Total.fillna('').str.match('^\d+\.?\d?$')].Total.value_counts()

0        229
600      184
500      179
520      167
570      138
        ... 
65.5       1
664        1
152        1
285.5      1
390.5      1
Name: Total, Length: 1117, dtype: int64

In [813]:
#Fix
def fix_total(df):
    df.Total.replace(['-', '0,0', '0', 'Out', 'Disq.', 'withdrawn', 'DSQ', 'Disq', 'BMB', 'BMB?', 'bmb', 'x', 'X', 'out'], np.nan, inplace = True)
    df.Total = df.Total.str.replace('?', '', regex = False)
    df.loc[df[df.Total.fillna('').str.contains("[a-zA-Z]")].index, 'Total'] = df[df.Total.fillna('').str.contains("[a-zA-Z]")].Total.str.split(' ', expand = True)[0].str.strip('RANWMY')
    df.Total = df.Total.str.replace(',', '.', regex = False)
    return df

##### Best columns

In [814]:
#Inspect
#df.BestSQ.value_counts()
#df.BestBP.value_counts()
df.BestDL.value_counts()
#df[df.BestSQ.fillna('').str.contains("x|X")].BestSQ.value_counts()
#df[df.BestBP.fillna('').str.contains("x|X")].BestBP.value_counts()
#df[df.BestDL.fillna('').str.contains("x|X")].BestDL.value_counts()
#df.BestSQ.str.strip('WNR ').value_counts()
#df.BestBP.str.strip('kgBMBWNR ').value_counts()
#df.BestDL.str.strip('WNRF ').value_counts()

-              80
240            59
220            56
230            54
200            53
250            41
210            39
190            38
185            35
235            34
180            33
160            32
225            30
140            25
175            25
245            24
170            24
260            23
120            23
150            23
130            22
205            22
155            21
215            21
280            20
270            19
255            17
100            16
115            15
125            15
195            13
165            13
242.5          13
145            12
x              12
95             11
265            11
135            11
252.5          11
142.5          11
90             10
222.5          10
207.5          10
275             9
285             9
65              9
110             9
0               9
92.5            9
237.5           9
132.5           8
127.5           8
227.5           8
162.5           7
320             7
152.5     

In [815]:
#Fix
def fix_bests(df):
    df.BestSQ.replace(['-', 'BMB', '0', '?', 'No attempt', '*', 'bmb'], np.nan, inplace = True)
    df.BestBP.replace(['-', 'x', 'BMB', '0', '?','260 BMB', 'No attempt', '*', 'bmb'], np.nan, inplace = True)
    df.BestDL.replace(['-', 'BMB', '0', '?', 'No attempt', '*', 'bmb'], np.nan, inplace = True)
    df.loc[df[df.BestSQ.fillna('').str.contains("x|X")].index, 'BestSQ'] = np.nan
    df.loc[df[df.BestBP.fillna('').str.contains("x|X")].index, 'BestBP'] = np.nan
    df.loc[df[df.BestDL.fillna('').str.contains("x|X")].index, 'BestDL'] = np.nan
    df.BestSQ = df.BestSQ.str.strip('WNR ')
    df.BestBP = df.BestBP.str.strip('kgBMBWNR ')
    df.BestDL = df.BestDL.str.strip('WNRF ')
    df.BestBP = df.BestBP.str.replace('/', ' ', regex = False)
    df.BestBP = df.BestBP.str.replace('o', '0', regex = False)
    df.BestDL = df.BestDL.str.replace('/', ' ', regex = False)
    df.loc[1862, 'BestDL'] = np.nan
    df.loc[df[df.BestBP.fillna('').str.contains(" ")].index, 'BestBP'] = df[df.BestBP.fillna('').str.contains(" ")].BestBP.str.split(' ', expand = True)[0]
    df.loc[df[df.BestDL.fillna('').str.contains(" ")].index, 'BestDL'] = df[df.BestDL.fillna('').str.contains(" ")].BestDL.str.split(' ', expand = True)[0]    
    return df

##### Attempt columns

In [816]:
#Inspect
col = 'DL3'
#df[col].value_counts().nlargest(100)
#df[col].value_counts().nsmallest(100)
#df[df[col].fillna('').str.match('.*[xXfF]$')][col].value_counts() #XxFf at the end
#df[df[col].fillna('').str.match('^[a-zA-Z]+$')][col].value_counts() #Made of letters
#df[df[col].fillna('').str.contains(',')][col].unique() #Contains comma as decimal
#df[~df[col].fillna('').str.match('^-?\d+\.?\d?$')][col].value_counts() #Not of the form ddd.ddd 
alls = []
for col in [i + j for i in ['SQ', 'BP', 'DL'] for j in ['1', '2', '3']]:
    alls.extend(df[~df[col].fillna('').str.match('[0-9.-]*')][col].unique())
set(alls)

set()

In [817]:
#Fix
def general_fix(df, col):
    df[col].replace(['8', '-', 'x', 'X', 'BMB', 'out', 'F', 'PASS', '??X', '150-x', '0,0', 'x?', '0', '1', '##',
                    '4th at', '?''??????n', 'BP & DL ONLY', 'Push Pull only', 'Withdrawn Injured', '_amp_nbsp_semi_', '`'], np.nan, inplace = True)
    df[col] = df[col].str.replace(',', '.', regex = False)
    df[col] = df[col].str.replace('*', '', regex = False)
    df[col] = df[col].str.replace('`', '', regex = False)
    df[col] = df[col].str.replace('?', '', regex = False)
    df[col] = df[col].str.replace('-or', '', regex = False)
    df[col] = df[col].str.replace('n', '', regex = False)
    df.loc[df[df[col].fillna('').str.match('.*\s?[xXfF]$')].index, col] = '-' + df[df[col].fillna('').str.contains('.*\s?[xXfF]$')][col].str.strip('xfXF ')
    df.loc[df[df[col].fillna('').str.match('^x\s?.*')].index, col] = '-' + df[df[col].fillna('').str.contains('^x\s?.*')][col].str.strip('x ')
    df[col].replace('', np.nan, inplace = True)
    df.loc[df[df[col].fillna('').str.contains(" ")].index, col] = df[df[col].fillna('').str.contains(" ")][col].str.split(' ', expand = True)[0]
    df.loc[df[df[col].fillna('').str.contains("[a-zA-Z]")].index, col] = df[df[col].fillna('').str.contains("[a-zA-Z]")][col].str.strip('ARSW')
    return df

def fix_attempts(df):
    df.loc[2491, 'BP2'] = '202.5'
    for col in [i + j for i in ['SQ', 'BP', 'DL'] for j in ['1', '2', '3']]:
        df = general_fix(df, col)
    return df

##### Data organization

In [ ]:
def restructure(df)
    df = df[['CompID','Name','Sex','DOB','BDW','Class','Division','SQ1','SQ2','SQ3', 'BestSQ','BP1','BP2','BP3','BestBP','DL1','DL2','DL3','BestDL','Total','Wilks','Place']]
    #df = df.convert_dtypes()
    return df

### Execute

In [824]:
#One function
def fix_all(df):
    df = fix_sex(df)
    df = fix_class(df)
    df = fix_dob(df)
    df = fix_division(df)
    df = fix_wilks(df)
    df = fix_bdw(df)
    df = fix_place(df)
    df = fix_name(df)
    df = fix_total(df)
    df = fix_bests(df)
    df = fix_attempts(df)
    df = restructure(df)
    return df
df = fix_all(pd.read_csv("../../Data/Tidy_Full.csv", index_col = 0))


In [825]:
df.to_csv("../../Data/Clean_Full.csv", index_label = 'Index')

# Testing area

In [850]:
df[df[['SQ1', 'SQ2', 'SQ3','BP1', 'BP2', 'BP3','DL1', 'DL2', 'DL3', 'BestSQ', 'BestBP', 'BestDL', 'Total', 'Wilks']].isnull().all(axis=1)][['CompID','Name']]

,CompID,Name
Index,,
57,2,Lex Dabwido
59,2,Anthony Inglese
184,4,Tony Burley
232,9,Ray Owen
234,9,Wayne Parnwell
281,13,Wayne Parnwell
354,17,Tony Somerville
406,21,Steven Goeldner
424,22,Robert Withers


In [853]:
df[df[['SQ1', 'SQ2', 'SQ3','BP1', 'BP2', 'BP3','DL1', 'DL2', 'DL3', 'BestSQ', 'BestBP', 'BestDL', 'Total', 'Wilks']].isnull().all(axis=1)].CompID.unique()

array([  2,   4,   9,  13,  17,  21,  22,  28,  29,  39,  50,  52,  55,
        65, 104, 110, 111, 135, 139, 140, 146, 167, 169, 192, 196, 197,
       199, 700, 704, 766, 918], dtype=int64)

In [873]:
df[['CompID','DOB','BDW','SQ1','SQ2','SQ3', 'BestSQ','BP1','BP2','BP3','BestBP','DL1','DL2','DL3','BestDL','Total','Wilks','Place']].astype(float)

TypeError: float() argument must be a string or a number, not 'NAType'

In [906]:
df[df.Sex == pd.NA]

,CompID,Name,Sex,DOB,BDW,Class,Division,SQ1,SQ2,SQ3,BestSQ,BP1,BP2,BP3,BestBP,DL1,DL2,DL3,BestDL,Total,Wilks,Place
Index,,,,,,,,,,,,,,,,,,,,,,
